## 객실의 사용 여부 관련 데이터
1. 데이터를 로드(hotel_bookings.csv)
2. 데이터에 대한 정보를 확인
3. 해당 데이터에서 문제가 있는 부분을 확인하여 수정

In [353]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [354]:
# data 폴더 안에 hotel_booking.csv 파일을 로드
hotel = pd.read_csv("../data_git/data/hotel_bookings.csv")
hotel.head()

,is_canceled,deposit_type,lead_time,stays_in_weekend_nights,stays_in_week_nights,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr
0,0,No Deposit,105.0,2,5,NaN,0,0,1,0,131.50
1,0,No Deposit,303.0,2,2,NaN,0,0,0,0,73.95
2,0,No Deposit,33.0,2,3,0.0,0,0,0,0,NaN
3,0,No Deposit,48.0,0,1,0.0,0,0,1,0,80.30
4,0,No Deposit,216.0,4,7,0.0,0,0,2,0,60.90


In [355]:
hotel.isna().sum()

is_canceled                          0
deposit_type                         0
lead_time                            5
stays_in_weekend_nights              0
stays_in_week_nights                 0
is_repeated_guest                  358
previous_cancellations               0
previous_bookings_not_canceled       0
booking_changes                      0
days_in_waiting_list                 0
adr                               1063
dtype: int64

In [356]:
# 결측치의 비율
print("lead_time column 결측치의 비율 : ", round(5 /len(hotel) * 100, 2), '%')
print("is_repeated_guest column 결측치의 비율 : ", round(358 /len(hotel) * 100, 2), '%')
print("adr column 결측치의 비율 : ", round(1063 /len(hotel) * 100, 2), '%')

lead_time column 결측치의 비율 :  0.03 %
is_repeated_guest column 결측치의 비율 :  1.79 %
adr column 결측치의 비율 :  5.32 %


- lead_time 컬럼의 결측치의 비율은 매우 작기 때문에 제거 -> 결측치인 인덱스를 제외
- is_reapted_guest 해당 데이터에서 개수가 많은 데이터로 결측치를 채워준다.
- adr 컬럼의 결측치는 해당 데이터를 확인하고 특정한 조건에 맞춰서 데이터를 채워준다.

- lead_time column

In [357]:
# subset으로 dropna에 조건을 넣을 수 있다.
hotel = hotel.dropna(subset=['lead_time'], axis = 0)

# hotel['lead_time'].loc[hotel['lead_time'].isna()]   #985, 1087, 4125, 4923, 16221
# hotel.drop([985, 1087, 4125, 4923, 16221])

- is_repeated_guest column

In [358]:
# is_repeated_guest 0 vs 1 확인
# hotel['is_repeated_guest'].sum()
hotel['is_repeated_guest'].value_counts()

is_repeated_guest
0.0    18888
1.0      749
Name: count, dtype: int64

In [435]:
hotel['is_repeated_guest'].fillna(0, inplace=True)

/var/folders/v0/r6b3g15j6gd1x8947g8ly2qr0000gn/T/ipykernel_50036/2360375739.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hotel['is_repeated_guest'].fillna(0, inplace=True)


- adr column

In [436]:
hotel['adr'].describe()

count    18931.000000
mean       101.416396
std         49.236271
min          0.000000
25%         68.840000
50%         94.500000
75%        126.000000
max        451.500000
Name: adr, dtype: float64

In [437]:
# 통계 정보를 확인하니 객실 요금 평균에 음수가 존재 -> 이상한 데이터
# 이상치 데이터는 제거
# 인덱스의 조건식을 생성 -> 객실 요금 데이터에서 0보다 작은
flag = hotel['adr'] < 0
hotel = hotel.loc[~flag]

In [440]:
# adr의 결측치들을 deposit_type의 값에 따라 그룹화를 하고 평균의 adr의 값을 채워준다.

# deposit_type에 따른 adr의 평균값을 확인
hotel.groupby('deposit_type')['adr'].mean().to_dict()
deposit_adr_mean = hotel.groupby('deposit_type')['adr'].mean().to_dict()

In [441]:
hotel.groupby('deposit_type')

In [442]:
test_hotel = hotel.copy()

In [443]:
null_flag = test_hotel['adr'].isna()
for key in deposit_adr_mean:
    # 인덱스 조건식
    flag = test_hotel['deposit_type'] == key
    # flag와 null_flag 둘다 만족하는 인덱스 필터
    test_hotel.loc[null_flag & flag, 'adr'] = deposit_adr_mean[key]

In [444]:
test_hotel['adr'].isna().sum()

np.int64(0)

In [445]:
null_flag = hotel['adr'].isna()
for key in deposit_adr_mean:
    # 인덱스 조건식
    flag = hotel['deposit_type'] == key
    # flag와 null_flag 둘다 만족하는 인덱스 필터
    hotel.loc[null_flag & flag, 'adr'] = deposit_adr_mean[key]

hotel['adr'].isna().sum()


np.int64(0)

In [446]:
hotel.isna().sum()

is_canceled                       0
deposit_type                      0
lead_time                         0
stays_in_weekend_nights           0
stays_in_week_nights              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
booking_changes                   0
days_in_waiting_list              0
adr                               0
dtype: int64

In [369]:
# # map(), apply()
# hotel['adr'].map(
#     lambda x : print(x)
# )

In [370]:
# # map(), apply()
# hotel['adr'].apply(
#     lambda x : print(x)
# )

- 1차원 시리즈 데이터에서 map과 series는 같은 역할을 한다

In [371]:
# hotel.map(
#     lambda x : print(x)
# )
# DataFrame에서 map()함수는 첫 번째 시리즈의 values를 모두 탐색하고 다음 시리즈의 value로 넘어간다

In [372]:
# hotel.apply(
#     lambda x : print(x)
# )

# DataFrame에서 apply 함수는 시리즈별로 탐색을 하는 함수
# 차원을 한 개 축소하여 데이터를 확인

In [409]:
# hotel['is_repeated_guest'] =hotel['is_repeated_guest'].astype(int)

In [448]:
# groupby() 함수와 apply()를 조합하여
# deposit_type에 따라서 adr의 결측치를 평균값으로 대체
adj_hotel = hotel.groupby('deposit_type').apply(
    lambda x : x.fillna(x.mean()))

In [449]:
# 멀티인덱스인 경우
# 특정 인덱스만 초기화하고 싶으면 reset_index()에서 매개변수 설정 변경
adj_hotel.reset_index(level='deposit_type', inplace = True)


In [450]:
adj_hotel['is_canceled'].value_counts

<bound method IndexOpsMixin.value_counts of 0        0
1        0
2        0
3        0
4        0
        ..
15249    0
15428    0
15661    0
16644    0
16851    0
Name: is_canceled, Length: 19994, dtype: int64>

In [ ]:
# hotel.loc[hotel['deposit_type'] == 'No Deposit', 'adr'].fillna(101.971806, inplace= True)

In [ ]:
# hotel.loc[hotel['deposit_type'] == 'Non Refund', 'adr'].fillna(89.689476, inplace= True)

In [ ]:
# hotel.loc[hotel['deposit_type'] == 'Refundable', 'adr'].fillna(67.456154, inplace = True)

In [ ]:
# hotel['adr']

In [413]:
adj_hotel.head()

,deposit_type,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr
0,No Deposit,0,105.0,2,5,0,0,0,1,0,131.500000
1,No Deposit,0,303.0,2,2,0,0,0,0,0,73.950000
2,No Deposit,0,33.0,2,3,0,0,0,0,0,101.971806
3,No Deposit,0,48.0,0,1,0,0,0,1,0,80.300000
4,No Deposit,0,216.0,4,7,0,0,0,2,0,60.900000


- 종속 변수 데이터의 균형 문제 발생 -> 약 7:1

In [414]:
adj_hotel['deposit_type'] = adj_hotel['deposit_type'].map(
    {
        'No Deposit' : 0,
        'Non Refund' : 1,
        'Refundable' : 2
    }
)

1. adj_hotel에서 독립변수와 종속변수로 데이터를 나눠준다.
2. trian, test롤 데이터를 분할(7:3)
3. 랜덤포레스트 분류모델을 생성
4. train 데이터를 이용하여 학습
5. test 데이터를 이용하여 예측
6. 평가지표 중 정확도를 확인

In [415]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import time

In [416]:
start = time.time()
x = adj_hotel.drop('is_canceled', axis = 1)
y = adj_hotel['is_canceled']
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y,
    test_size=0.3,
    random_state=42,
    stratify = y
)
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)
pred = clf.predict(X_test)
end = time.time()

In [417]:
# sampling 안 한 경우
print(end - start)
cm = confusion_matrix(pred, Y_test)
acc = accuracy_score(pred, Y_test)

print(cm)
print("정확도 : ", round(acc, 2))

# 분류 보고서 출력
print(classification_report(pred, Y_test))
# 코드의 진행 시간 
print("코드 진행 시간 : ", end - start)

0.8810782432556152
[[5221  429]
 [  58  291]]
정확도 :  0.92
              precision    recall  f1-score   support

           0       0.99      0.92      0.96      5650
           1       0.40      0.83      0.54       349

    accuracy                           0.92      5999
   macro avg       0.70      0.88      0.75      5999
weighted avg       0.95      0.92      0.93      5999

코드 진행 시간 :  0.8810782432556152


In [451]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [452]:
ros = RandomOverSampler()

In [453]:
x_ros, y_ros = ros.fit_resample(x, y)

In [466]:
Counter(y_ros)

Counter({0: 17594, 1: 17594})

In [467]:
start = time.time()

X_train_ros, X_test_ros, Y_train_ros, Y_test_ros = train_test_split(
    x_ros, y_ros,
    test_size=0.3,
    random_state=42,
    stratify = y_ros
)
clf = RandomForestClassifier()
clf.fit(X_train_ros, Y_train_ros)
pred_ros = clf.predict(X_test_ros)
end = time.time()

In [468]:
# randomoversampler 사용한 경우
print(end - start)
cm_ros= confusion_matrix(pred_ros, Y_test_ros)
acc_ros = accuracy_score(pred_ros, Y_test_ros)

print(cm_ros)
print("정확도 : ", round(acc_ros, 2))

# 분류 보고서 출력
print(classification_report(pred_ros, Y_test_ros))
# 코드의 진행 시간 
print("RandomOver 데이터의 소요 시간 : ", end - start)

1.4349710941314697
[[4977   32]
 [ 302 5246]]
정확도 :  0.97
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5009
           1       0.99      0.95      0.97      5548

    accuracy                           0.97     10557
   macro avg       0.97      0.97      0.97     10557
weighted avg       0.97      0.97      0.97     10557

RandomOver 데이터의 소요 시간 :  1.4349710941314697


In [424]:
from imblearn.over_sampling import SMOTE

In [469]:
smote = SMOTE()

In [470]:
x_sm, y_sm = smote.fit_resample(x, y)

In [471]:
Counter(y_sm)

Counter({0: 17594, 1: 17594})

In [472]:
start = time.time()

X_train_sm, X_test_sm, Y_train_sm, Y_test_sm = train_test_split(
    x_sm, y_sm,
    test_size=0.3,
    random_state=42,
    stratify = y_sm
)
clf = RandomForestClassifier()
clf.fit(X_train_sm, Y_train_sm)
pred = clf.predict(X_test_sm)
end = time.time()

In [473]:
# smote 사용한 경우
print(end - start)
cm = confusion_matrix(pred, Y_test)
acc = accuracy_score(pred, Y_test)

print(cm)
print("정확도 : ", round(acc, 2))

# 분류 보고서 출력
print(classification_report(pred, Y_test))
# 코드의 진행 시간 
print("코드 진행 시간 : ", end - start)

1.8078820705413818
[[4682  623]
 [ 597 4655]]
정확도 :  0.88
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      5305
           1       0.88      0.89      0.88      5252

    accuracy                           0.88     10557
   macro avg       0.88      0.88      0.88     10557
weighted avg       0.88      0.88      0.88     10557

코드 진행 시간 :  1.8078820705413818
